In [5]:
import requests
import time
import json
from pprint import pprint

base_url = 'http://10.55.72.205:8000'
base_url = 'http://localhost:8001'
api_key = '8f3aec8e-b1e4-4967-8fc9-3b073edf0e89'
api_key = '1800f748-a688-49c3-ad00-f660d40fd2e3'
algorithm_name = 'Bronchus Segmentation'
api_dicom_location = '{0}/api/dicomlocation'.format(base_url)
api_dataset = '{0}/api/dataset'.format(base_url)
api_dataset_ready = '{0}/api/dataset/ready'.format(base_url)
api_data_object = '{0}/api/dataobject'.format(base_url)
api_trigger = '{0}/api/trigger'.format(base_url)
api_algorithm = '{0}/api/algorithm'.format(base_url)

In [6]:
# Check if we already have the Orthanc Dicom Location, if not lets add it
orthanc_location = None
r = requests.get(api_dicom_location, headers={'API_KEY': api_key})
for d in r.json():
    if d['name'] == 'Orthanc':
        orthanc_location = d
        break
        
if not orthanc_location:
    # If we didn't find the Orthanc location then lets add it now
    data = {'name': 'Orthanc',
           'host': '127.0.0.1',
           'port': 4242,
           'ae_title': 'ORTHANC',
           'move_ae_title': 'LOCALDICOM',
           'move_port': 7778}
    r = requests.post(api_dicom_location, headers={'API_KEY': api_key}, data=data)
    
    if r.status_code >= 200 and r.status_code < 300:
        print('Added Orthanc Location')
        orthanc_location = r.json()
        
pprint(orthanc_location)

Added Orthanc Location
{'ae_title': 'ORTHANC',
 'host': '127.0.0.1',
 'id': 1,
 'name': 'Orthanc',
 'owner_key': '1800f748-a688-49c3-ad00-f660d40fd2e3',
 'port': 4242}


In [7]:
# Create a new Dataset
data = {'from_dicom_location': orthanc_location['id']}
dataset = None
r = requests.post(api_dataset, headers={'API_KEY': api_key}, data=data)
if r.status_code >= 200 and r.status_code < 300:
        print('Added Dataset')
        dataset = r.json()
        
pprint(dataset)

Added Dataset
{'from_dicom_location': {'ae_title': 'ORTHANC',
                         'host': '127.0.0.1',
                         'id': 1,
                         'name': 'Orthanc',
                         'owner_key': '1800f748-a688-49c3-ad00-f660d40fd2e3',
                         'port': 4242},
 'from_dicom_location_id': 1,
 'id': 1,
 'input_data_objects': [],
 'output_data_objects': [],
 'owner_key': '1800f748-a688-49c3-ad00-f660d40fd2e3',
 'timeout': '2019-05-10T03:45:24.604',
 'timestamp': '2019-05-09T03:45:24.613',
 'to_dicom_location': None,
 'to_dicom_location_id': None}


In [10]:
# Add a Dicom file to the dataset
# Create a new Dataset
data = {'dataset': dataset['id'],
       'type': 'DICOM',
       'dicom_retrieve': 'MOVE',
       'seriesUID': '2.16.840.1.114362.1.11775105.22396782581.502959996.669.2'}
data_object = None
r = requests.post(api_data_object, headers={'API_KEY': api_key}, data=data)
if r.status_code >= 200 and r.status_code < 300:
        print('Added Data Object')
        data_object = r.json()
        
pprint(r.json())

Added Data Object
{'children': [],
 'dataset_id': 1,
 'id': 3,
 'is_fetched': False,
 'is_input': True,
 'is_sent': False,
 'meta_data': None,
 'parent': None,
 'parent_id': None,
 'path': None,
 'series_instance_uid': '2.16.840.1.114362.1.11775105.22396782581.502959996.669.2',
 'status': None,
 'timestamp': '2019-05-09T03:46:31.676',
 'type': 'DICOM'}


In [11]:
# Get the algorithm and the default settings
algorithm = None
r = requests.get(api_algorithm, headers={'API_KEY': api_key})
if r.status_code == 200:
    for a in r.json():
        if algorithm_name in a['name']:
            algorithm = a
        print(a)

{'name': 'Bronchus Segmentation', 'default_settings': {'contours': [], 'radiomics': {}, 'pyradiomics_settings': {'binWidth': 25, 'resampledPixelSpacing': None, 'interpolator': 'sitkNearestNeighbor', 'verbose': True, 'removeOutliers': 10000}, 'resample_to_image': False, 'append_histogram': False, 'histogram_bins': 256}}


In [12]:
# Trigger the algorithm with our dataset containing the data object
data={'dataset': dataset['id'],
     'algorithm': algorithm['name']}
r = requests.post(api_trigger, headers={'API_KEY': api_key}, data=data)

if r.status_code == 200:
    # Poll the URL given to determine the progress of the task
    poll_url = '{0}{1}'.format(base_url, r.json()['poll'])
    
    while(1):
        r = requests.get(poll_url, headers={'API_KEY': api_key})
        status = r.json()
        print(status)

        if status['state'] == 'SUCCESS' or status['state'] == 'FAILURE':
            break

        time.sleep(1)
else:
    print(r.json())
    
print('Algorithm Processing Complete')

{'Error': 'Dataset contains Data Objects that have not been fetched yet. Wait for them to be fetched or DICOM Send them if necessary.'}
Algorithm Processing Complete


In [12]:
# Fetch the latest dataset to see the output objects and download the Nifti file!
r = requests.get('{0}/{1}'.format(api_dataset, dataset['id']), headers={'API_KEY': api_key})
if r.status_code == 200:
    dataset = r.json()
    pprint(dataset)

    for d in dataset['output_data_objects']:
        if d['path'].endswith('nii.gz'):
            #print(d)
            r = requests.get('http://localhost:8000/api/dataobject/download/{0}'.format(d['id']), headers={'API_KEY': api_key})
            filename = r.headers['Content-Disposition'].split('filename=')[1]
            print('Downloading to: {0}'.format(filename))
            open(filename, 'wb').write(r.content)

{'from_dicom_location': {'ae_title': 'ORTHANC',
                         'host': '127.0.0.1',
                         'id': 2,
                         'move_ae_title': 'LOCALDICOM',
                         'move_port': 7778,
                         'name': 'Orthanc',
                         'owner_key': '8f3aec8e-b1e4-4967-8fc9-3b073edf0e89',
                         'port': 4242,
                         'query': None,
                         'query_class': None},
 'from_dicom_location_id': 2,
 'id': 44,
 'input_data_objects': [{'dataset': None,
                         'dataset_id': 44,
                         'id': 123,
                         'is_fetched': True,
                         'is_input': True,
                         'is_sent': False,
                         'meta_data': None,
                         'parent': None,
                         'parent_id': None,
                         'path': '/tmp/tmpl6jumgup/2.16.840.1.114362.1.11775105.22396782581.502959996.

In [219]:
r = requests.get('{0}/{1}'.format(api_dataset_ready, dataset['id']), headers={'API_KEY': api_key})
r.json()

{'ready': True}